In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-2.4.7'
# spark_version = 'spark-2.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,876

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://usc-bootcamp-yelpreview-text-analysis.s3.us-east-2.amazonaws.com/reviews.csv"
spark.sparkContext.addFile(url)
panda_df = spark.read.csv(SparkFiles.get("reviews.csv"), sep=",", header=True)

# Show DataFrame
panda_df.show(40)

+--------------------+-------------+------------+
|             reviews|       rating|review_count|
+--------------------+-------------+------------+
|Panda Express was...|5 star rating|          63|
|The dude and I ca...|5 star rating|          63|
|I ordered 5 total...|1 star rating|          63|
|I always order Pa...|3 star rating|          63|
|Decided to try Pa...|5 star rating|          63|
|I've never had a ...|4 star rating|          63|
|The family meal d...|1 star rating|          63|
|Quality has sever...|2 star rating|          63|
|Paid for a bowl a...|1 star rating|          63|
|Order a bowl with...|1 star rating|          63|
|Went through the ...|1 star rating|          63|
|When I think of p...|2 star rating|          63|
|Horrible is a und...|1 star rating|          63|
|Yes the drive thr...|2 star rating|          63|
|Okay..so Panda is...|4 star rating|          63|
|Going through Dri...|2 star rating|          63|
|My entrees were a...|2 star rating|          63|


In [20]:
# panda_df=panda_df.withColumn('ratings', panda_df['rating'])
# panda_df.show()

+---+--------------------+-------------+------------+-------------+
|_c0|             reviews|       rating|review_count|      ratings|
+---+--------------------+-------------+------------+-------------+
|  0|Panda Express was...|5 star rating|          63|5 star rating|
|  1|The dude and I ca...|5 star rating|          63|5 star rating|
|  2|I ordered 5 total...|1 star rating|          63|1 star rating|
|  3|I always order Pa...|3 star rating|          63|3 star rating|
|  4|Decided to try Pa...|5 star rating|          63|5 star rating|
|  5|I've never had a ...|4 star rating|          63|4 star rating|
|  6|The family meal d...|1 star rating|          63|1 star rating|
|  7|Quality has sever...|2 star rating|          63|2 star rating|
|  8|Paid for a bowl a...|1 star rating|          63|1 star rating|
|  9|Order a bowl with...|1 star rating|          63|1 star rating|
| 10|Went through the ...|1 star rating|          63|1 star rating|
| 11|When I think of p...|2 star rating|        

In [23]:
# Change number of star rating to "good", "decent", "poor"
import pyspark.sql.functions as F
panda_df = panda_df.withColumn("ratings",
      F.when((panda_df.rating == "1 star rating") | (panda_df.rating == "2 star rating"), 'poor')
      .when((panda_df.rating == "3 star rating") | (panda_df.rating == "4 star rating"), 'decent')
      .otherwise('good'))
panda_df.show()


+---+--------------------+-------------+------------+-------+
|_c0|             reviews|       rating|review_count|ratings|
+---+--------------------+-------------+------------+-------+
|  0|Panda Express was...|5 star rating|          63|   good|
|  1|The dude and I ca...|5 star rating|          63|   good|
|  2|I ordered 5 total...|1 star rating|          63|   poor|
|  3|I always order Pa...|3 star rating|          63| decent|
|  4|Decided to try Pa...|5 star rating|          63|   good|
|  5|I've never had a ...|4 star rating|          63| decent|
|  6|The family meal d...|1 star rating|          63|   poor|
|  7|Quality has sever...|2 star rating|          63|   poor|
|  8|Paid for a bowl a...|1 star rating|          63|   poor|
|  9|Order a bowl with...|1 star rating|          63|   poor|
| 10|Went through the ...|1 star rating|          63|   poor|
| 11|When I think of p...|2 star rating|          63|   poor|
| 12|Horrible is a und...|1 star rating|          63|   poor|
| 13|Yes

In [24]:
baba

+---+--------------------+-------------+------------+-------+------+
|_c0|             reviews|       rating|review_count|ratings|length|
+---+--------------------+-------------+------------+-------+------+
|  0|Panda Express was...|5 star rating|          63|   good|   334|
|  1|The dude and I ca...|5 star rating|          63|   good|   770|
|  2|I ordered 5 total...|1 star rating|          63|   poor|   154|
|  3|I always order Pa...|3 star rating|          63| decent|   634|
|  4|Decided to try Pa...|5 star rating|          63|   good|   262|
|  5|I've never had a ...|4 star rating|          63| decent|   644|
|  6|The family meal d...|1 star rating|          63|   poor|   129|
|  7|Quality has sever...|2 star rating|          63|   poor|   350|
|  8|Paid for a bowl a...|1 star rating|          63|   poor|   158|
|  9|Order a bowl with...|1 star rating|          63|   poor|   151|
| 10|Went through the ...|1 star rating|          63|   poor|   678|
| 11|When I think of p...|2 star r

In [25]:
# Clean dataframe
data_df = data_df['_c0', 'reviews', 'ratings', 'length']
data_df.show()

+---+--------------------+-------+------+
|_c0|             reviews|ratings|length|
+---+--------------------+-------+------+
|  0|Panda Express was...|   good|   334|
|  1|The dude and I ca...|   good|   770|
|  2|I ordered 5 total...|   poor|   154|
|  3|I always order Pa...| decent|   634|
|  4|Decided to try Pa...|   good|   262|
|  5|I've never had a ...| decent|   644|
|  6|The family meal d...|   poor|   129|
|  7|Quality has sever...|   poor|   350|
|  8|Paid for a bowl a...|   poor|   158|
|  9|Order a bowl with...|   poor|   151|
| 10|Went through the ...|   poor|   678|
| 11|When I think of p...|   poor|   272|
| 12|Horrible is a und...|   poor|   481|
| 13|Yes the drive thr...|   poor|   312|
| 14|Okay..so Panda is...| decent|  1541|
| 15|Going through Dri...|   poor|   657|
| 16|My entrees were a...|   poor|   171|
| 17|I'm done being ri...|   poor|  1067|
| 18|Worst Panda in Fr...|   poor|   417|
| 19|I've been in here...|   good|   410|
+---+--------------------+-------+

### Feature Transformations


In [26]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
label_encoder = StringIndexer(inputCol='ratings',outputCol='label')
tokenizer = Tokenizer(inputCol="reviews", outputCol="tokens")
stop_word_remover = StopWordsRemover(inputCol='tokens',outputCol='filtered_tokens')
hasher = HashingTF(inputCol="filtered_tokens", outputCol='hashed_tokens')
idf = IDF(inputCol='hashed_tokens', outputCol='idf_tokens')


In [27]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
vectorizer = VectorAssembler(inputCols=['idf_tokens', 'length'], outputCol='features')

In [28]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[label_encoder, tokenizer, stop_word_remover, hasher, idf, vectorizer])


In [29]:
# Fit and transform the pipeline
pipeline_model = pipeline.fit(data_df)
cleaned_df = pipeline_model.transform(data_df)

In [30]:
# Show label and resulting features
cleaned_df.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262145,[2711,610...|
|  1.0|(262145,[9090,131...|
|  0.0|(262145,[4402,355...|
|  2.0|(262145,[329,2325...|
|  1.0|(262145,[8086,335...|
|  2.0|(262145,[78,8287,...|
|  0.0|(262145,[19153,49...|
|  0.0|(262145,[16595,28...|
|  0.0|(262145,[15664,20...|
|  0.0|(262145,[5381,172...|
|  0.0|(262145,[2325,736...|
|  0.0|(262145,[78,2437,...|
|  0.0|(262145,[4402,137...|
|  0.0|(262145,[6796,172...|
|  2.0|(262145,[2437,755...|
|  0.0|(262145,[2410,124...|
|  0.0|(262145,[16091,31...|
|  0.0|(262145,[8985,134...|
|  0.0|(262145,[1536,232...|
|  1.0|(262145,[553,1489...|
+-----+--------------------+
only showing top 20 rows



In [34]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned_df.randomSplit([0.75, 0.25], seed=42)

# Create a Naive Bayes model and fit training data
model = NaiveBayes()
predictor = model.fit(training)

In [35]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

+-----+--------------------+-------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  _c0|             reviews|ratings|length|label|              tokens|     filtered_tokens|       hashed_tokens|          idf_tokens|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+-------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    1|The dude and I ca...|   good|   770|  1.0|[the, dude, and, ...|[dude, came, pand...|(262144,[9090,131...|(262144,[9090,131...|(262145,[9090,131...|[-3265.0250599721...|[1.0,3.7281047288...|       0.0|
|   10|Went through the ...|   poor|   678|  0.0|[went, through, t...|[went, drive-thro...|(262144,[2325,736...|(262144,[2325,736...|(262145,[2325,736...|[-2129.5432463

In [36]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.665399
